In [1]:
from gensim.models import Word2Vec
import pickle
import numpy as np
import pandas as pd
import os
from gensim.test.utils import get_tmpfile
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from gensim.parsing.preprocessing import preprocess_string
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import strip_numeric
def remove_ip(s):
    # Replace all ip adresses with '<ip>' tag
    ip_regexp = r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b"
    return re.sub(ip_regexp, '<ip>', s)
def remove_email(s):
    # Replace all email adresses with '<email>' tag
    email_regexp = r"([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})"
    return re.sub(email_regexp, '<email>', s)
def remove_mailto(s):
    # Replace all "<mailto:<email>>" with <email>. Email adresses should be replaced by remove_email first.
    return s.replace("<mailto:<email>>", "<email>")
def remove_url(s):
    # Replace all url's with '<url>' tag
    url_regexp = r"((http|ftp|https):\/\/)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)"
    s = re.sub(url_regexp, '<url>', s)
    # Sometimes url's are inside <> so we need to replace <<url>> with <url>
    return s.replace("<<url>>", "<url>")
def remove_punc(s, exceptions):
    # Remove all punctuation from string with exceptions in list exceptions
    remove = string.punctuation
    for exception in exceptions:
        remove = remove.replace(exception, "")
    # Create the pattern
    pattern = r"[{}]".format(remove)

    return re.sub(pattern, "", s)
def remove_custom_stopwords(s, stopwords):
    for stopword in stopwords:
        s = s.replace(stopword, "")
    return s
def lower_case(s):
    return s.lower()
def preprocess_sentence_fn(s):
    # Preprocess a single sentence to a list of tokens
    punc_exceptions = ['<', '>']
    custom_stopwords = ['dear', 'sincerely', 'thanks', 'yours', 'regards']
    filters = [lower_case,
               remove_ip,
               remove_email,
               remove_mailto,
               #remove_url,
               lambda x: remove_punc(x, punc_exceptions),
               remove_stopwords,
               lambda x: remove_custom_stopwords(x, custom_stopwords),
               strip_multiple_whitespaces,
               stem_text,
               strip_numeric]
    out = preprocess_string(s, filters=filters)
    return out
def preprocess_docs_fn(docs):
    # Apply preprocess_sentence_fn to a list of sentances (docs) to get a list of lists
    return [preprocess_sentence_fn(s) for s in docs]

In [3]:
# Read in the data
ticket_dat = pd.read_csv('../../data/12-04-ticket_dat.csv')
faq_dat = pd.read_csv('../../data/12-04-faq_dat.csv')
# Replace the NaNs
ticket_dat.fillna('', inplace=True)
faq_dat.fillna('', inplace=True)

In [4]:
# FAQ question
faq_ques = list(faq_dat.question)
n_faq_ques = len(faq_ques)
# FAQ answer
faq_ans = list(faq_dat.answer_title + " " + faq_dat.answer)
n_faq_ans = len(faq_ans)
#ticket question
ticket_ques = list(ticket_dat.question)
n_ticket_ques = len(ticket_ques)
#ticket ans
ticket_ans = list(ticket_dat.answer)
n_ticket_ans = len(ticket_ans)

In [5]:
# Model assumption: same embedding for all
all_docs = faq_ques + faq_ans + ticket_ques + ticket_ans
# Model assumption: two different embeddings
all_ans = faq_ans + ticket_ans

In [6]:
# create a dictionary storing the cut points for the four datasets so we can re-split them after.
# use like all_docs[id_dict['faq_ques']] to get all faq questions.
id_dict = {
    'faq_ques': range(0, n_faq_ques),
    'faq_ans': range(n_faq_ques, n_faq_ques + n_faq_ans),
    'ticket_ques': range(n_faq_ques + n_faq_ans, n_faq_ques + n_faq_ans + n_ticket_ques),
    'ticket_ans': range(n_faq_ques + n_faq_ans + n_ticket_ques, n_faq_ques + n_faq_ans + n_ticket_ques + n_ticket_ans)
}
all_docs_sep = {
    'faq_ques': faq_ques,
    'faq_ans': faq_ans,
    'ticket_ques': ticket_ques,
    'ticket_ans': ticket_ans}

In [7]:
all_docs_prepro = preprocess_docs_fn(all_docs)
all_ans_prepro = preprocess_docs_fn(all_ans)

In [159]:
#FOR DEBUGGING PURPOSE

#check if datasets contain empty strings
faq_ques_prepro = preprocess_docs_fn(faq_ques)
for i in range(len(faq_ques_prepro)):
    if not faq_ques_prepro[i]:
        print('faq question {}'.format(i))

In [8]:
#ALL ANSWERs
# checking if embedding model already exists
exists = os.path.isfile('../../code/embedding/models/word2vec_ans.model')
if exists:
    print('Word2vec embedding model already existing')
# Create word embedding model
else:
    print('Training word2vec on all answers')
    word_path = "../../code/embedding/models/word2vec_ans.model"
    word_tempfile = get_tmpfile(word_path)
    word_model = Word2Vec(all_ans_prepro, size=128, window=5, min_count=1, workers=4)
    word_model.save(word_path)

Word2vec embedding model already existing


In [9]:
ticket_ques_prepro = preprocess_docs_fn(ticket_ques)

In [10]:
#TICKET QUESTIONS
exists = os.path.isfile('../../code/embedding/models/word2vec_ticket_ques.model')
if exists:
    print('Word2vec embedding model already existing')
else:
    #not checking if already exists because if the first doesn't this won't either
    print('Training word2vec on ticket questions')
    word_path = "../../code/embedding/models/word2vec_ticket_ques.model"
    word_tempfile = get_tmpfile(word_path)
    word_model = Word2Vec(ticket_ques_prepro, size=128, window=5, min_count=1, workers=4)
    word_model.save(word_path)

Training word2vec on ticket questions


c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


# SIMILARITY

In [16]:
print('Loading Word2vec model')
model_path = '../../code/embedding/models/word2vec_ans.model'
model = Word2Vec.load(model_path)

Loading Word2vec model


In [17]:
def doc_emb(dat):
    mean_ans = np.empty((len(dat), 128), dtype=float)
    for j in range(len(dat)):
        sentence = dat[j]
        words = np.empty((len(sentence), 128), dtype=float)
        for i in range(len(sentence)):
            words[i] = model[sentence[i]]
        mean_ans[j] = np.apply_along_axis(np.mean, 0, words)
    return mean_ans

In [18]:
mean_ticket_ans = doc_emb(all_ans_prepro[len(faq_ans):len(all_ans)])
#create doc vector for faq ans i.e. average over each faq ans the word2vec vector for each word
mean_faq_ans = doc_emb(all_ans_prepro[0:len(faq_ans)])

c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [19]:
sim_matrix = cosine_similarity(mean_faq_ans, mean_ticket_ans)

In [20]:
#most similar faq - ticket mapping
FAQ_per_ticket = np.argmax(sim_matrix, axis=0)
strength_FAQ_ticket = np.max(sim_matrix, axis=0)

#small similarities are set to a separate class
thres = 0.2
FAQ_per_ticket[strength_FAQ_ticket < thres] = -1

In [55]:
#debug: print the vectors with 0 

for i in range(len(mean_ticket_ans)):
    zero = np.count_nonzero(mean_ticket_ans[i])
    if zero != 128: 
        print(i)
        
for i in range(len(mean_faq_ans)):
    zero = np.count_nonzero(mean_ticket_ans[i])
    if zero != 128: 
        print(i)

NameError: name 'mean_ticket_ans' is not defined

# CLASSIFICATION

In [21]:
# Load the Word2Vec model
model_path = '../../code/embedding/models/word2vec_ticket_ques.model'
model = Word2Vec.load(model_path)

In [30]:
def doc_emb(dat):
    mean_ans = np.empty((len(dat), 128), dtype=float)
    for j in range(len(dat)):
        sentence = dat[j]
        words = np.empty((len(sentence), 128), dtype=float)
        for i in range(len(sentence)):
            print(sentence[i])
            words[i] = model[sentence[i]]
        mean_ans[j] = np.apply_along_axis(np.mean, 0, words)
    return mean_ans

In [31]:
ticket_question_embeddings = doc_emb(ticket_ques_prepro)

ladi
gentlemen
week
left
regist
listen
work
empa
belong
eth
domain
finish
bsc
chemistri
eth
zurich
login
data
usernam
password
log
mystudi
kind
daniel
aeppli
daniel
aeppli
lindenstrass
pfäffikon
zhhello
mr
greute
thank
detail
lindenstrass
pfäffikon
zh
address
eggbrunnenweg
russikon
best
daniel
aeppli
march
joel
greuter
wrote
ticket
ideth
<<email>>
hello
mr
greuter
chang
password
log
moment
send
new
password
sm
daniel
aeppli
march
daniel
aeppli
wrote
<<email>>
pelyuan
zhang
contact
traffic
analysi
detect
sign
compromis
malwar
trojan
virus
accord
benutzerordnung
für
telematik
bot
forbidden
connect
infect
comput
eth
network
clean
infect
httpsrechtssammlungspethzchdokumenteenpdf
analysi
data
traffic
show
follow
signatur
event
etpro
trojan
networmwinkoobfaceght
ping
date
sourc
<ip>
dest
<ip>
event
etpro
trojan
networmwinkoobfaceght
ping
date
sourc
<ip>
dest
<ip>
event
etpro
trojan
networmwinkoobfaceght
ping
date
sourc
<ip>
dest
<ip>
event
etpro
trojan
networmwinkoobfaceght
ping
date
sourc
<

c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  



advanc
time
actuari
apv
eth
like
send
mail
pharmaci
student
like
pick
limit
mail
hour
best
anna
tekako
actuarin
apv
ladi
gentlemen
week
longer
log
mail
server
outlook
webmail
login
work
problem
tri
fix
account
outlook
set
complet
base
instruct
httpswwwethzchcontentdamethzassociatesservicesserviceitservicesfil
catalogueemailcalendarmailboxmailclientoutlookdepdfhttpswwwethzchcontentdamethzassociatesservicesserviceitservicesfilescatalogueemailcalendar
mailboxmailclientoutlookenpdf
manual
input
port
come
window
password
input
enter
correct
password
window
appear
new
newer
manual
chang
configur
best
lilian
müller
httpswwwethzchcontentdamethzassociatesservicesserviceitservicesfilescatalogueemailcalendarmailboxmailclientoutlookenpdf
httpswwwethzchcontentdamethzassociatesservicesserviceitservicesfilescatalogueemailcalendarmailboxmailclientoutlookenpdf
mr
greuter
tip
tri
password
remain
enter
unfortun
login
work
us
follow
set
server
mailethzch
usernam
d
\
lilmuel
password
best
lilian
müller
mr

wish
shinichi
um
schrieb
joel
greuter
ticketsystem
ideth
<<email>>
>
mr
prof
dr
shinichi
sunagawa
thethank
mr
friederich
longer
necessari
monica
kohnthank
mkhoi
ruedi
technician
greet
marinamr
lichtensteig
info
bit
fix
technician
contact
lg
monica
kohn
monica
kohn
eth
zurich
power
electron
system
laboratori
secretariat
etl
h
physikstrass
zurich
switzerland
<email>
wwwpeseeethzch
phone
fax
hall
lichtensteig
technician
info
monica
kohnthank
mr
lichtsteig
monica
kohnhello
mr
lichtensteig
technician
yesterdai
morn
h
minut
site
pack
come
morn
printer
run
printer
offlin
pe
offic
soon
printer
start
run
inform
kind
monica
kohnhello
mr
lichtensteig
technician
return
compani
good
minut
expect
morn
monica
kohnhello
mr
lichtensteig
technician
faigl
left
printer
run
thank
support
lg
monica
kohn
monica
kohn
eth
zurich
power
electron
system
laboratori
secretariat
etl
h
physikstrass
zurich
switzerland
<email>
wwwpeseeethzch
phone
fax
concern
forgotten
password
ethi
login
usernam
asookdeo
email
<email>

quick
feedback
best
cécile
imhofhello
mr
greuter
list
meetingsciencec
<email>
meetingsciencecityextern
<email>
mutat
best
cécile
imhof
mr
greuter
thank
august
quotmoderationquot
replac
ulrik
von
groll
isabel
stöckli
list
best
cécile
imhofhello
mr
greuter
thank
august
notic
worri
bit
regist
extern
newslett
gmx
address
follow
error
messagehello
mr
greuter
problem
gmx
address
fix
thank
quick
feedback
best
cécile
imhofhello
mr
tuduc
thank
inform
provid
hotmail
gmail
set
look
forward
decis
best
cécile
imhofhello
christian
mean
switch
spf
sympa
gener
notic
problem
gmxch
todai
gener
us
spf
greet
cristianhello
mr
tuduc
start
commun
work
fall
program
soon
newslett
registr
work
soon
possibl
tell
problem
gmx
address
fix
contact
sympa
administr
directli
best
cécile
imhof
hello
backup
team
want
client
pw
data
reset
patrick
stocker
gave
guid
mr
joergpet
lorch
us
command
backup
archiv
us
command
line
client
gui
item
util
quotchang
passwordquot
quotdelet
filespacesquot
patrick
stocker
unfortun
error
m

...

password
eappli
portal
work
mailbox
servic
situat
thank
best
elizaveta
tretiakova
ye
like
password
sent
post
birthdat
address
elizaveta
tretiakova
uetlibergstrass
a
zurich
instruct
includ
letter
right
confus
password
usag
password
thank
best
elizaveta
tretiakova
thu
sep
joel
greuter
ticketsystem
ideth
<
<email>>
wrote
team
possibl
request
storag
space
email
inbox
im
member
board
architektura
fachverein
der
architekturstudierenden
answer
email
delet
right
awai
best
jasmin
art
sent
iphon
recordingthank
best
eth
zurich
id
procur
amp
portfolio
jürg
häubi
head
contract
order
manag
stb
j
stampfenbachstrass
ch
zurich
<email>
wwwidethzch
phone
mobil
sir
madam
niko
alimperti
student
cbb
master
dai
problem
email
platform
mail
remain
space
delet
quarter
email
mail
larg
volum
space
problem
warn
exist
help
problem
thank
time
best
niko
alimperti
hello
everybodi
im
eth
alumni
student
current
attend
languag
cours
languag
center
semest
materi
provid
olat
unfortun
access
unfortun
login
work
desir
help
b

dai
technic
user
bauibib
lock
colleagu
enter
password
incorrectli
time
releas
account
ps
account
releas
intervent
certain
time
best
eth
zurich
roberto
pascolo
manag
ibk
hil
e
stefanofransciniplatz
ch
zürich
good
dai
print
hg
e
unfortun
result
good
printer
make
lot
stain
left
ink
residu
previou
page
make
funni
beep
sound
mayb
good
come
look
printer
best
alexandra
fuch
sent
iphon
good
dai
user
like
extend
dai
guest
hire
process
delai
dai
guest
creat
user
pamplonag
extend
dai
receiv
follow
text
«hello
open
daili
guest
account
gustavo
pamplona
​​becaus
creation
employ
contract
requir
time
expect
possibl
extend
access
dai
migrat
offic
vaud
migrat
offic
zh
hr
involv
access
data
sent
thank
understand
good
even
greet
maya
kamber
quotif
question
aris
possibl
dispos
best
fiona
kühne
oper
window
problem
descript
hello
backup
team
creat
backupnodesccerrbloch
backupwindow
dai
greet
leandra
altern
email
adresscli
mean
sccerrblochethzch
greet
leandra
ladi
gentlemen
studi
eth
physic
student
uzh
semest

christma
holidai
cool
work
best
remo
eth
zurich
financ
amp
control
depart
account
sew
d
scheuchzerstrass
ch
zurich
<email>
websit
financ
amp
control
ladi
gentlemen
coincid
store
udriv
pdf
file
adob
suddenli
file
quot
pdfquot
open
document
warn
messag
appear
«the
connect
secur
»
pretti
strang
fix
best
brigitt
schiesser
ladi
gentlemen
thank
email
im
awai
happi
urgent
case
colleagu
colleagu
legal
servic
happi
help
brigitt
schiesser
legal
servic
eth
zurich
mr
locher
thank
feedback
current
absent
colleagu
nadin
stadelmann
know
avail
tuesdai
best
brigitt
schiesser
mr
locher
need
process
request
colleagu
welcom
come
best
nadin
stadelmann
ladi
gentlemen
try
switch
wifi
connect
privat
laptop
eduroam
eduroam
receiv
follow
error
messag
mayb
help
convers
android
phone
work
flawlessli
thank
best
kristina
hipp
mirel
kisic
thank
quick
feedback
sorri
renew
demand
mean
click
quotconnectquot
ignor
warn
kristina
hipp
mirel
kisic
ticket
ideth
wrotethank
mirel
kisic
ticket
ideth
wrote
hi
everybodi
iv
acces

kind
gregori
rozanski
mirel
kisic
thank
quick
repli
tri
suggest
got
error
messag
sai
permiss
access
share
error
messag
attach
great
help
solv
issu
advanc
gregori
rozanski
mirel
kisic
sorri
email
tri
work
thank
help
kind
gregori
rozanski
hello
technic
problem
printer
pavilion
build
hönggerberg
printer
displai
error
messag
sc
printer
em
ask
send
soon
possibl
lot
yvo
corpataux
oper
mac
os
io
internet
connect
problem
descript
shanghai
oct
need
vpn
eth
altern
email
adresshoi
mirel
realiz
china
block
vpn
tunnel
secret
vpn
work
china
respons
vpn
best
christophervpn
block
eth
zurich
china
abl
prove
quotgreat
china
firewallquot
block
access
vpn
concentr
httpssslvpnethzch
dai
receiv
desper
support
request
eth
member
current
china
specif
user
current
china
travel
implement
emerg
resolut
problem
short
notic
todai
stopgap
solut
support
circumvent
chines
censorship
current
legal
clear
remain
feasibl
know
long
last
goe
undetect
current
document
spread
solut
avail
affect
support
correct
function
verif

new
quota
gb
drive
\\
nasethzch
\
mavtinspirenasus
new
quota
gb
thank
kind
josef
meil
ladi
gentlemen
id
servic
desk
receiv
lot
mail
im
board
member
amiv
exam
prepar
cours
write
mail
student
iv
extens
muck
mailbox
possibl
increas
storag
space
best
julia
jäggi
sir
madam
need
semest
lectur
linear
algebra
matlab
order
success
connect
vpn
soon
want
download
product
link
error
messag
appear
problem
websit
softwareethzch
thank
help
paul
och
sirmadam
student
uzh
appli
special
student
eth
chang
password
yesterdai
record
log
todai
tri
method
faq
password
didnt
work
write
email
ask
help
jin
wang
login
wanjin
mobil
number
look
forward
respons
thank
jin
wang
samuel
fux
cluster
support
im
current
wait
pend
job
leonhard
cluster
execut
reasin
want
kill
run
job
afternoon
command
bkill
jobid
job
hour
later
run
job
queue
start
run
mayb
help
issu
best
tobia
stegemann
urban
borstnik
thank
help
possibl
cluster
busi
moment
beacus
job
pend
queue…
best
tobia
stegemann
beschreibunghello
toshiba
notebook
late
ti

In [25]:
print('Running CV on Classifier...')
classifier_CV = RandomForestClassifier()
scores = cross_val_score(classifier_CV, ticket_question_embeddings, FAQ_per_ticket, cv=5)
cv_score = scores.mean()

Running CV on Classifier...


c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimat

In [29]:
print('Training Classifier...')
classifier = RandomForestClassifier()
classifier.fit(X=ticket_question_embeddings, y=FAQ_per_ticket)
#dump(classifier, 'classifier/models/RF_word2vec.joblib')
train_score = classifier.score(X=ticket_question_embeddings, y=FAQ_per_ticket)

print('Training Score: {0} \n Cross Val Score: {1}'.format(train_score, cv_score))
print('###############')

Training Classifier...


c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training Score: 0.9871611982881597 
 Cross Val Score: 0.21132894467194116
###############
